In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%matplotlib inline
import torch
import numpy as np

knock70

In [4]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [5]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import KeyedVectors

d = 300
L = 4

def preprc(data):
  x = []
  y = []
  label = {"b":0, "e":2, "t":1, "m":3} #ビジネス:0, エンターテインメント:2, 科学技術:1, 健康:3
  for l in data:
    da = l.split("\t")
    # titleを数字をすべて0、アルファベットを小文字にする
    xd = [i.rstrip("\n").strip(":'$%?,.()!;").strip('"') for i in re.sub("[0-9]+", "0", da[1]).lower().split(" ") if i != "" and i != "...\n"]
    emb_sum = torch.zeros((d,))
    nd = 0
    for w in xd:
      if not w in model.key_to_index:
        nd += 1
        #print(w)
      else:
        emb_sum += torch.from_numpy(model[w])
    if (len(xd) - nd) != 0:
      x.append(emb_sum / (len(xd) - nd))
      y.append(label[da[0]])
  return torch.stack(x), torch.tensor(y)

train = []
valid = []
test = []
with open('/content/drive/MyDrive/train.txt', encoding='utf-8') as f:
  for l in f:
    train.append(l)
with open('/content/drive/MyDrive/valid.txt', encoding='utf-8') as f:
  for l in f:
    valid.append(l)
with open('/content/drive/MyDrive/test.txt', encoding='utf-8') as f:
  for l in f:
    test.append(l)
X_train, Y_train = preprc(train)
X_valid, Y_valid = preprc(valid)
X_test, Y_test = preprc(test)

<ipython-input-5-7aa742f11a13>:24: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  emb_sum += torch.from_numpy(model[w])


In [ ]:
np.save('/content/drive/MyDrive/X_train.npy', X_train.numpy())
np.save('/content/drive/MyDrive/Y_train.npy', Y_train.numpy())
np.save('/content/drive/MyDrive/X_valid.npy', X_valid.numpy())
np.save('/content/drive/MyDrive/Y_valid.npy', Y_valid.numpy())
np.save('/content/drive/MyDrive/X_test.npy', X_test.numpy())
np.save('/content/drive/MyDrive/Y_test.npy', Y_test.numpy())

knock71

In [6]:
def read_f():
  X_train = torch.from_numpy(np.load('/content/drive/MyDrive/X_train.npy'))
  Y_train = torch.from_numpy(np.load('/content/drive/MyDrive/Y_train.npy'))
  X_valid = torch.from_numpy(np.load('/content/drive/MyDrive/X_valid.npy'))
  Y_valid = torch.from_numpy(np.load('/content/drive/MyDrive/Y_valid.npy'))
  X_test = torch.from_numpy(np.load('/content/drive/MyDrive/X_test.npy'))
  Y_test = torch.from_numpy(np.load('/content/drive/MyDrive/Y_test.npy'))
  return X_train, Y_train, X_valid, Y_valid, X_test, Y_test

X_train, Y_train, X_valid, Y_valid, X_test, Y_test = read_f()
W = torch.randn(d, L, requires_grad=True)

In [7]:
yh1 = torch.softmax(X_train[0] @ W, dim=-1)
Yh = torch.softmax(X_train[:4] @ W, dim=-1)
print(f"{yh1}")
print(f"{Yh}")

tensor([0.4267, 0.0905, 0.0809, 0.4019], grad_fn=<SoftmaxBackward0>)
tensor([[0.4267, 0.0905, 0.0809, 0.4019],
        [0.8286, 0.0388, 0.0763, 0.0563],
        [0.9470, 0.0331, 0.0113, 0.0086],
        [0.6877, 0.0637, 0.1995, 0.0491]], grad_fn=<SoftmaxBackward0>)


knock72

In [8]:
from torch import nn

class SLPN(nn.Module):
  def __init__(self, ins, outs):
    super().__init__()
    self.fc = nn.Linear(ins, outs, bias=False)
    nn.init.normal_(self.fc.weight, 0.0, 1.0)
  def forward(self, x):
    return self.fc(x)

n_W = SLPN(d, L)

criterion = torch.nn.CrossEntropyLoss()
l1 = criterion(n_W(X_train[0]), Y_train[0])
n_W.zero_grad()
l1.backward()
print(f"x1のクロスエントロピー損失：{l1}")
print(f"x1の行列Wに対する勾配：\n{n_W.fc.weight.grad}")
l = criterion(n_W(X_train[:4]), Y_train[:4])
n_W.zero_grad()
l.backward()
print(f"\n事例集合x1,x2,x3,x4のクロスエントロピー損失：{l}")
print(f"事例集合x1,x2,x3,x4の行列Wに対する勾配：\n{n_W.fc.weight.grad}")

x1のクロスエントロピー損失：0.5464903116226196
x1の行列Wに対する勾配：
tensor([[-6.0336e-03,  9.8539e-03,  2.9576e-05,  ..., -3.7536e-03,
          1.2119e-02,  6.4914e-03],
        [-2.0806e-03,  3.3979e-03,  1.0199e-05,  ..., -1.2944e-03,
          4.1789e-03,  2.2384e-03],
        [ 1.5071e-02, -2.4614e-02, -7.3879e-05,  ...,  9.3762e-03,
         -3.0271e-02, -1.6215e-02],
        [-6.9572e-03,  1.1362e-02,  3.4104e-05,  ..., -4.3282e-03,
          1.3974e-02,  7.4851e-03]])

事例集合x1,x2,x3,x4のクロスエントロピー損失：2.2436132431030273
事例集合x1,x2,x3,x4の行列Wに対する勾配：
tensor([[ 0.0076,  0.0066, -0.0047,  ..., -0.0067, -0.0493,  0.0182],
        [ 0.0106, -0.0361, -0.0030,  ..., -0.0130,  0.0154,  0.0225],
        [ 0.0040,  0.0126,  0.0127,  ...,  0.0374,  0.0221, -0.0514],
        [-0.0222,  0.0169, -0.0050,  ..., -0.0177,  0.0118,  0.0106]])


knock73

In [9]:
optimizer = torch.optim.SGD(n_W.parameters(), lr=0.1)

for epoch in range(100):
  n_W.train()
  loss_train = 0.0
  for i, nt in enumerate(X_train):
    la = Y_train[i]
    optimizer.zero_grad()
    loss = criterion(n_W(nt), la)
    loss.backward()
    optimizer.step()
    loss_train += loss.item()
  loss_train = loss_train / (i + 1)
  n_W.eval()
  with torch.no_grad():
    loss_valid = criterion(n_W(X_valid), Y_valid)
  if (epoch + 1) % 10 == 0:
    print(f"epoch: {epoch+1}, loss_train: {loss_train}, loss_valid: {loss_valid}")

epoch: 10, loss_train: 0.30510667339374276, loss_valid: 0.35849931836128235
epoch: 20, loss_train: 0.29205016401022427, loss_valid: 0.3647209107875824
epoch: 30, loss_train: 0.28775366595812407, loss_valid: 0.3708248436450958
epoch: 40, loss_train: 0.28575683409073416, loss_valid: 0.3753372132778168
epoch: 50, loss_train: 0.28466064087373716, loss_valid: 0.3786197006702423
epoch: 60, loss_train: 0.28399469973118774, loss_valid: 0.38104507327079773
epoch: 70, loss_train: 0.2835611868486161, loss_valid: 0.382874459028244
epoch: 80, loss_train: 0.2832645015908309, loss_valid: 0.3842821419239044
epoch: 90, loss_train: 0.28305363124811167, loss_valid: 0.3853852152824402
epoch: 100, loss_train: 0.2828993544906188, loss_valid: 0.3862644135951996


knock74

In [10]:
def cal_acc(model, x, y):
  model.eval()
  cor = 0
  with torch.no_grad():
    for i, input in enumerate(x):
      pred = torch.argmax(n_W(input), dim=-1)
      cor += (pred == y[i]).sum().item()
  return cor / len(x)

acc_train = cal_acc(n_W, X_train, Y_train)
acc_test = cal_acc(n_W, X_test, Y_test)
print(f"学習データの正解率：{acc_train}")
print(f"評価データの正解率：{acc_test}")

学習データの正解率：0.9036967711745437
評価データの正解率：0.8779026217228464


knock75

In [ ]:
def cal_loss(model, x, y, criterion):
  model.eval()
  loss = 0.0
  cor = 0
  with torch.no_grad():
    for i, input in enumerate(x):
      loss += criterion(model(input), y[i]).item()
  return loss / len(x)

n_W = SLPN(d, L)
optimizer = torch.optim.SGD(n_W.parameters(), lr=0.1)
losss_train = []
accs_train = []
losss_valid = []
accs_valid =[]
for epoch in range(100):
  n_W.train()
  for i, nt in enumerate(X_train):
    la = Y_train[i]
    optimizer.zero_grad()
    loss = criterion(n_W(nt), la)
    loss.backward()
    optimizer.step()
  losss_train.append(cal_loss(n_W, X_train, Y_train, criterion))
  accs_train.append(cal_acc(n_W, X_train, Y_train))
  losss_valid.append(cal_loss(n_W, X_valid, Y_valid, criterion))
  accs_valid.append(cal_acc(n_W, X_valid, Y_valid))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(tight_layout=True)
plt.subplot(121)
plt.plot(range(1,101), losss_train, label="train")
plt.plot(range(1,101), losss_valid, label="valid")
plt.xlabel("epoch")
plt.ylabel('loss')
plt.legend()
plt.subplot(122)
plt.plot(range(1,101), accs_train, label="train")
plt.plot(range(1,101), accs_valid, label="valid")
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()
plt.show()

knock76

In [15]:
n_W = SLPN(d, L)
optimizer = torch.optim.SGD(n_W.parameters(), lr=0.1)
losss_train = []
accs_train = []
losss_valid = []
accs_valid =[]
with open('k76.txt', 'w') as f:
  for epoch in range(100):
    n_W.train()
    for i, nt in enumerate(X_train):
      la = Y_train[i]
      optimizer.zero_grad()
      loss = criterion(n_W(nt), la)
      loss.backward()
      optimizer.step()
    f.write(f'{epoch+1}\n{n_W.state_dict()}\n{optimizer.state_dict()}')

knock77

In [16]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

class NewsDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    return [self.X[idx], self.y[idx]]

dataset_train = NewsDataset(X_train, Y_train)
dataset_valid = NewsDataset(X_valid, Y_valid)
dataset_test = NewsDataset(X_test, Y_test)

dataloader_train = DataLoader(dataset_train, batch_size=1, shuffle=True)
dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=len(dataset_test), shuffle=False)

In [17]:
import time

def cal_laa(model, criterion, loader):
  model.eval()
  loss = 0.0
  total = 0
  correct = 0
  with torch.no_grad():
    for inputs, labels in loader:
      outputs = model(inputs)
      loss += criterion(outputs, labels).item()
      pred = torch.argmax(outputs, dim=-1)
      total += len(inputs)
      correct += (pred == labels).sum().item()

  return loss / len(loader), correct / total

def train_model(dataset_train, dataset_valid, batch_size, model, criterion, optimizer, num_epochs):
  dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
  dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)

  log_train = []
  log_valid = []
  for epoch in range(num_epochs):
    s_time = time.time()

    model.train()
    for inputs, labels in dataloader_train:
      optimizer.zero_grad()

      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

    loss_train, acc_train = cal_laa(model, criterion, dataloader_train)
    loss_valid, acc_valid = cal_laa(model, criterion, dataloader_valid)
    log_train.append([loss_train, acc_train])
    log_valid.append([loss_valid, acc_valid])

    torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, f'checkpoint{epoch + 1}.pt')

    e_time = time.time()

    print(f'epoch: {epoch + 1}, loss_train: {loss_train:.4f}, accuracy_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f}, accuracy_valid: {acc_valid:.4f}, {(e_time - s_time):.4f}sec')

  return {'train': log_train, 'valid': log_valid}

n_W = SLPN(300, 4)
optimizer = torch.optim.SGD(n_W.parameters(), lr=1e-1)

for batch_size in [2 ** i for i in range(11)]:
  print(f'バッチサイズ: {batch_size}')
  log = train_model(dataset_train, dataset_valid, batch_size, n_W, criterion, optimizer, 1)

バッチサイズ: 1
epoch: 1, loss_train: 0.3976, accuracy_train: 0.8618, loss_valid: 0.4316, accuracy_valid: 0.8376, 6.1901sec
バッチサイズ: 2
epoch: 1, loss_train: 0.3689, accuracy_train: 0.8727, loss_valid: 0.4072, accuracy_valid: 0.8428, 4.4221sec
バッチサイズ: 4
epoch: 1, loss_train: 0.3597, accuracy_train: 0.8776, loss_valid: 0.4003, accuracy_valid: 0.8488, 1.7390sec
バッチサイズ: 8
epoch: 1, loss_train: 0.3564, accuracy_train: 0.8782, loss_valid: 0.3968, accuracy_valid: 0.8481, 1.0233sec
バッチサイズ: 16
epoch: 1, loss_train: 0.3543, accuracy_train: 0.8797, loss_valid: 0.3953, accuracy_valid: 0.8518, 0.6081sec
バッチサイズ: 32
epoch: 1, loss_train: 0.3533, accuracy_train: 0.8800, loss_valid: 0.3947, accuracy_valid: 0.8533, 0.4413sec
バッチサイズ: 64
epoch: 1, loss_train: 0.3529, accuracy_train: 0.8801, loss_valid: 0.3945, accuracy_valid: 0.8533, 0.3566sec
バッチサイズ: 128
epoch: 1, loss_train: 0.3530, accuracy_train: 0.8802, loss_valid: 0.3943, accuracy_valid: 0.8525, 0.2494sec
バッチサイズ: 256
epoch: 1, loss_train: 0.3530, accuracy_

knock78

In [1]:
def train_model(dataset_train, dataset_valid, batch_size, model, criterion, optimizer, num_epochs):
  dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
  dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)

  log_train = []
  log_valid = []
  for epoch in range(num_epochs):
    s_time = time.time()

    model.train()
    for inputs, labels in dataloader_train:
      optimizer.zero_grad()

      inputs = inputs.to(device)
      labels = labels.to(device)
      outputs = model.forward(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

    loss_train, acc_train = cal_laa(model, criterion, dataloader_train, device)
    loss_valid, acc_valid = cal_laa(model, criterion, dataloader_valid, device)
    log_train.append([loss_train, acc_train])
    log_valid.append([loss_valid, acc_valid])

    torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, f'checkpoint{epoch + 1}.pt')

    e_time = time.time()

    print(f'epoch: {epoch + 1}, loss_train: {loss_train:.4f}, accuracy_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f}, accuracy_valid: {acc_valid:.4f}, {(e_time - s_time):.4f}sec')

  return {'train': log_train, 'valid': log_valid}

knock79

In [2]:
model = SLPN(300, 4)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

for batch_size in [2 ** i for i in range(11)]:
  print(f'バッチサイズ: {batch_size}')
  log = train_model(dataset_train, dataset_valid, batch_size, model, criterion, optimizer, 1)

NameError: ignored